In [1]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    host="jx5gomuqu1.us-east-1.aws.clickhouse.cloud",
    port=8443,
    username="default",
    password="LujWuvKhdIKn",
)
# client = clickhouse_connect.get_client(host='127.0.0.1', port=8123)

In [4]:
client.command("CREATE TABLE test_table (key UInt16, value String) ENGINE Memory")

''

In [5]:
data = [[100, "value1"], [200, "value2"]]
client.insert("test_table", data)
print(client.query("SELECT * FROM test_table").result_set)

[(100, 'value1'), (200, 'value2')]


In [2]:
import pandas as pd

df = pd.read_parquet("./dbtest.parquet")

In [4]:
query = (
    "create table tiles ("
    + ",".join([f"{c} Float32" for c in df.columns])
    + ") engine = MergeTree() order by orig_lam"
)
query

'create table tiles (alpha0 Float32,orig_lam Float32,B_lam_0 Float32,B_lam_1 Float32,B_lam_2 Float32,B_lam_3 Float32,B_lam_4 Float32,B_lam_5 Float32,B_lam_6 Float32,B_lam_7 Float32,B_lam_8 Float32,B_lam_9 Float32,B_lam_10 Float32,B_lam_11 Float32,B_lam_12 Float32,B_lam_13 Float32,B_lam_14 Float32,B_lam_15 Float32,B_lam_16 Float32,B_lam_17 Float32,B_lam_18 Float32,B_lam_19 Float32,B_lam_20 Float32,B_lam_21 Float32,B_lam_22 Float32,B_lam_23 Float32,B_lam_24 Float32,B_lam_25 Float32,B_lam_26 Float32,B_lam_27 Float32,B_lam_28 Float32,B_lam_29 Float32,B_lam_30 Float32,B_lam_31 Float32,B_lam_32 Float32,B_lam_33 Float32,B_lam_34 Float32,B_lam_35 Float32,B_lam_36 Float32,B_lam_37 Float32,B_lam_38 Float32,B_lam_39 Float32,B_lam_40 Float32,B_lam_41 Float32,B_lam_42 Float32,B_lam_43 Float32,B_lam_44 Float32,B_lam_45 Float32,B_lam_46 Float32,B_lam_47 Float32,B_lam_48 Float32,B_lam_49 Float32,twb_min_lam Float32,twb_mean_lam Float32,twb_max_lam Float32,sim_sizes Float32,todo Float32,grid_pt_idx Flo

In [5]:
client.command(query)

Code: 57. DB::Exception: Table default.tiles already exists. (TABLE_ALREADY_EXISTS) (version 22.12.1.593 (official build))



DatabaseError: :HTTPDriver for http://127.0.0.1:8123 returned response code 500)
 Code: 57. DB::Exception: Table default.tiles already exists. (TABLE_ALREADY_EXISTS) (version 22.12.1.593 (official build))


In [15]:
client.insert_df("tiles", df)

In [17]:
%%time
client.insert_df("tiles", df[:400000])

CPU times: user 7.02 s, sys: 175 ms, total: 7.2 s
Wall time: 7.43 s


In [3]:
import pyarrow

tbl = pyarrow.Table.from_pandas(df[:5000000])

In [4]:
%%time
client.insert_arrow("tiles", tbl)

CPU times: user 1.64 s, sys: 3.43 s, total: 5.07 s
Wall time: 2min 17s


In [17]:
df.shape[0] / (7 * 60 + 43.8)

41846.2462268219

In [16]:
%%time
sel_df = client.query_df("select * from tiles order by orig_lam limit 10000")
sel_df.shape

CPU times: user 42.6 ms, sys: 6.79 ms, total: 49.4 ms
Wall time: 79.6 ms


(10000, 69)

In [39]:
sel_df.shape[0] * sel_df.shape[1] * 4 / 1e6

2.76

In [17]:
%%time
client.query(
    "select " + ",".join([f"min(B_lam_{i})" for i in range(50)]) + " from tiles"
).result_set

CPU times: user 2.24 ms, sys: 1.14 ms, total: 3.38 ms
Wall time: 405 ms


[(0.06528124213218689,
  0.064506895840168,
  0.06400763243436813,
  0.06356178969144821,
  0.06519397348165512,
  0.06459040194749832,
  0.06479469686746597,
  0.06528934836387634,
  0.06237378343939781,
  0.06019483134150505,
  0.06254054605960846,
  0.06508979201316833,
  0.06514236330986023,
  0.06412892043590546,
  0.06313243508338928,
  0.0608912855386734,
  0.06583709269762039,
  0.06441992521286011,
  0.0632510855793953,
  0.06558826565742493,
  0.06099971383810043,
  0.06388067454099655,
  0.0623963437974453,
  0.06425051391124725,
  0.062368571758270264,
  0.06508255749940872,
  0.06364434957504272,
  0.06259889900684357,
  0.06259263306856155,
  0.06526196002960205,
  0.06184881925582886,
  0.06611113250255585,
  0.06383296102285385,
  0.06484857946634293,
  0.06429416686296463,
  0.06435570865869522,
  0.06498193740844727,
  0.06387448310852051,
  0.06395195424556732,
  0.0639881044626236,
  0.06390348821878433,
  0.05955151468515396,
  0.06256946921348572,
  0.065037578344